# Imports and configs

In [1]:
!pip install -q autogluon.tabular ray==2.10.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.1/65.1 MB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.2/352.2 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.2/266.2 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.1/64.1 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.2/68.2 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 105.6 MB/s eta 0:00:00


In [2]:
from autogluon.tabular import TabularPredictor
import pandas as pd
import numpy as np
import shutil

In [3]:
class CFG:
    train_path = '/kaggle/input/ai-competition-africa/train.csv'
    test_path = '/kaggle/input/ai-competition-africa/test.csv'
    sample_sub_path = '/kaggle/input/ai-competition-africa/sample_submission.csv'
    original_data_path = '/kaggle/input/wine-quality-dataset/WineQT.csv'
    
    target = 'quality'
    seed = 1807
    time_limit = 3600 * 10

# Loading data and predefining folds

In [4]:
train = pd.read_csv(CFG.train_path, index_col='id')
test = pd.read_csv(CFG.test_path, index_col='id')
original = pd.read_csv(CFG.original_data_path, index_col='Id')

train[CFG.target] = train[CFG.target].astype(int)
original[CFG.target] = original[CFG.target].astype(int)

mapping = {3: 0, 4: 1, 5: 2, 6: 3, 7: 4, 8: 5}
train[CFG.target] = train[CFG.target].map(mapping)
original[CFG.target] = original[CFG.target].map(mapping)

# Fitting the predictor

In [5]:
predictor = TabularPredictor(
    problem_type='multiclass',
    eval_metric='quadratic_kappa',
    label=CFG.target,
    verbosity=2
)

No path specified. Models will be saved in: "AutogluonModels/ag-20241222_234319"


In [6]:
predictor.fit(
    train_data=train,
    pseudo_data=original,
    time_limit=CFG.time_limit,
    excluded_model_types = ["KNN"],
    presets='best_quality',
    ag_args_fit={
        'num_gpus': 1, 
        'num_cpus': 4
    }
)

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Nov 10 10:07:59 UTC 2024
CPU Count:          4
Memory Avail:       30.26 GB / 31.35 GB (96.5%)
Disk Space Avail:   19.50 GB / 19.52 GB (99.9%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the data. Then holdout validation data is used to detect stacked 

(_ray_fit pid=5239) [1000]	valid_set's multi_logloss: 1.05245	valid_set's quadratic_kappa: 0.393563


(_ray_fit pid=5287) 	Training S1F2 with GPU, note that this may negatively impact model quality compared to CPU training.
(_ray_fit pid=5335) 	Training S1F3 with GPU, note that this may negatively impact model quality compared to CPU training.


(_ray_fit pid=5335) [1000]	valid_set's multi_logloss: 1.07425	valid_set's quadratic_kappa: 0.367485


(_ray_fit pid=5383) 	Training S1F4 with GPU, note that this may negatively impact model quality compared to CPU training.
(_ray_fit pid=5431) 	Training S1F5 with GPU, note that this may negatively impact model quality compared to CPU training.
(_ray_fit pid=5479) 	Training S1F6 with GPU, note that this may negatively impact model quality compared to CPU training.


...

(_ray_fit pid=14312) 	Training S1F7 with GPU, note that this may negatively impact model quality compared to CPU training.


(_ray_fit pid=14312) [1000]	valid_set's multi_logloss: 1.07117	valid_set's quadratic_kappa: 0.364714
(_ray_fit pid=14312) [2000]	valid_set's multi_logloss: 1.07575	valid_set's quadratic_kappa: 0.376177


(_ray_fit pid=14360) 	Training S1F8 with GPU, note that this may negatively impact model quality compared to CPU training.


(_ray_fit pid=14360) [1000]	valid_set's multi_logloss: 1.06879	valid_set's quadratic_kappa: 0.348573


(_dystack pid=163) 	0.3676	 = Validation score   (quadratic_kappa)
(_dystack pid=163) 	582.63s	 = Training   runtime
(_dystack pid=163) 	5.24s	 = Validation runtime
(_dystack pid=163) Fitting model: RandomForest_r39_BAG_L1 ... Training model for up to 1065.66s of the 4065.67s of remaining time.
(_dystack pid=163) 	0.362	 = Validation score   (quadratic_kappa)
(_dystack pid=163) 	6.99s	 = Training   runtime
(_dystack pid=163) 	0.51s	 = Validation runtime
(_dystack pid=163) Fitting model: CatBoost_r167_BAG_L1 ... Training model for up to 1057.85s of the 4057.86s of remaining time.
(_dystack pid=163) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (1 workers, per: cpus=4, gpus=1, memory=0.06%)
(_ray_fit pid=14435) 	Training S1F1 with GPU, note that this may negatively impact model quality compared to CPU training.
(_ray_fit pid=14435) 	Warning: CatBoost on GPU is experimental. If you encounter issues, use CPU for training CatBoost instead.
(_ray_fit p

In [7]:
predictor.leaderboard(silent=True).style.background_gradient(subset=['score_val'], cmap='RdYlGn')

,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3,0.426727,quadratic_kappa,14.005610,3053.373359,0.003922,3.352286,3,True,178
1,NeuralNetTorch_r158_BAG_L2,0.426670,quadratic_kappa,14.001688,3050.021073,0.932082,182.145587,2,True,158
2,NeuralNetTorch_r31_BAG_L2,0.418169,quadratic_kappa,14.115807,2972.517471,1.046202,104.641985,2,True,172
3,WeightedEnsemble_L2,0.416423,quadratic_kappa,0.236574,433.933955,0.002924,2.901876,2,True,109
4,NeuralNetTorch_r30_BAG_L2,0.415115,quadratic_kappa,14.032553,2984.248399,0.962948,116.372914,2,True,137
5,NeuralNetTorch_r79_BAG_L2,0.414954,quadratic_kappa,14.002809,2986.036168,0.933204,118.160682,2,True,122
6,NeuralNetTorch_r41_BAG_L2,0.414911,quadratic_kappa,14.037230,2978.974339,0.967625,111.098853,2,True,155
7,NeuralNetTorch_r143_BAG_L2,0.414842,quadratic_kappa,14.022222,3000.650091,0.952617,132.774606,2,True,169
8,NeuralNetTorch_r87_BAG_L1,0.414445,quadratic_kappa,0.108150,186.312119,0.108150,186.312119,1,True,70
9,NeuralNetTorch_r86_BAG_L2,0.413632,quadratic_kappa,14.021412,2966.226710,0.951807,98.351225,2,True,139


# Creating a submission file

In [8]:
test_pred_probs = predictor.predict_proba(test)
sub = pd.read_csv(CFG.sample_sub_path)
sub[CFG.target] = np.argmax(test_pred_probs, axis=1)
sub[CFG.target] = sub[CFG.target].map({0: 3, 1: 4, 2: 5, 3: 6, 4: 7, 5: 8})
sub.to_csv('sub_autogluon.csv', index=False)
sub.head()

,id,quality
0,15000,5
1,15001,5
2,15002,6
3,15003,6
4,15004,5


In [9]:
shutil.rmtree("AutogluonModels")